In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt

In [1]:
from package.src.loading import DataLoader
loader = DataLoader('./data/kidney_disease.csv')
loader.load(target_column="classification")

In [9]:
X, y = loader.X_train, loader.y_train
from package.src.preprocessing import FeatureSelector, DataPreprocessor, DataNormalizer

pp = DataPreprocessor()
fs = FeatureSelector()
X, y = pp.transform(X,y)
fs.pca_fit(X)

rbc
pc
pcc
ba
pcv
wc
rc
htn
dm
cad
appet
pe
ane
classification
age
bp
sg
al
su
rbc
pc
pcc
ba
bgr
bu
sc
sod
pot
hemo
pcv
wc
rc
htn
dm
cad
id
age
bp
sg
al
su
rbc
pc
pcc
ba
bgr
bu
sc
sod
pot
hemo
pcv
wc
rc
htn
dm
('no', '\tno')
('\tyes', ' yes')
('\tyes', 'yes')
(' yes', 'yes')
cad
('no', '\tno')
appet
pe
ane
classification
('ckd', 'ckd\t')
25 componenents (PCA)


In [ ]:
from sklearn.preprocessing import LabelEncoder,OrdinalEncoder,OneHotEncoder
le = LabelEncoder()
oe = OrdinalEncoder()
df = data[categorical_features+numerical_features]
df['target'] = data['classification']
df[categorical_features] = oe.fit_transform(data[categorical_features])
df[numerical_features] = data[numerical_features]
df['target'] = le.fit_transform(df['target'])
features = categorical_features + numerical_features